In [ ]:
# https://www.youtube.com/watch?v=HjvYsUL8NZQ&list=PLmQAMKHKeLZ-iTT-E2kK9uePrJ1Xua9VL&index=10
# https://codeshare.io/RbPONQ

In [ ]:
# !pip install chromadb openai langchain tiktoken

In [ ]:
# !pip show chromadb

In [ ]:
!wget -q https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip -P work

In [ ]:
!unzip -q work/new_articles.zip -d work/new_articles

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
loader = DirectoryLoader("work/new_articles/", glob="./*.txt", loader_cls=TextLoader)
document = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
text = text_splitter.split_documents(document)
len(text[1].page_content)

# Create DB

In [ ]:
from langchain import embeddings

In [ ]:
persist_directory = "work/db"
embedding = OpenAIEmbeddings()
vectorDb = Chroma.from_documents(documents=text, embedding=embedding, persist_directory=persist_directory)
vectorDb

# Make retriever

In [ ]:
retriever = vectorDb.as_retriever()
retriever

In [ ]:
docs = retriever.get_relevant_documents("How much money did Microsoft raise?")
docs

In [ ]:
retriever = vectorDb.as_retriever(search_kwargs={"k": 2})
retriever
docs = retriever.get_relevant_documents("How much money did Microsoft raise?")
docs

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
llm = OpenAI()
llm

In [ ]:
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)
qa_chain

In [ ]:
def process_llm_response(llm_response):
    print(llm_response)
    print('\n\nSources: ')
    for source in llm_response["source_documents"]:
        print(source.metadata["source"])

In [ ]:
query = "How much money did Microsoft raise?"
llm_response = qa_chain(query)
llm_response

In [ ]:

process_llm_response(llm_response=llm_response)

# Delete DB

In [ ]:
vectorDb.delete_collection()
vectorDb.persist()